In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import numpy as np
import tqdm as tqdm
import torch.optim as optim
import os


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\austi\CS\Austin_chess\myenv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\austi\CS\Austin_chess\myenv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\austi\CS\Austin_chess\myenv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "c:

Kaggle Data set
https://www.kaggle.com/datasets/ronakbadhe/chess-evaluations

In [2]:
df = pd.read_csv('Data/chessData.csv')

12958035
809877


In [3]:
print(df.head())
df = df.dropna()
print(len(df))

                                                 FEN Evaluation
0  rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...        -10
1  rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...        +56
2  rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBN...         -9
3  rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPP2PPP/RNBQKB...        +52
4  rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPPN1PPP/R1BQK...        -26
809877


In [4]:
def clean_evals(entry:str):
    if entry[0] == '#':
        return int(entry[1:]) + 10000
    else:
        return int(entry)



In [5]:
df['Evaluation'] = df['Evaluation'].apply(clean_evals)

In [6]:
def convert_to_index(pos: str) -> int:
    if pos == "-":
        return 0
    file_map = {value: key for key, value in enumerate("abcdefgh")}
    col = file_map[pos[0]]
    row = int(pos[1]) - 1
    # print(f"col: {col}, row: {row}")
    return 1 << (row * 8) + col


In [7]:

# Convert binary vectors to PyTorch tensors
# tensor_data = torch.tensor(binary_vectors, dtype=torch.float32)
def convert_pieces_to_vectors(piece_map: dict[str: int]) -> None:
    for piece, board in piece_map.items():
        piece_map[piece] = np.array([(board >> i) & 1 for i in range(64)], dtype=np.float32)

In [8]:

def convert_fen(fen: str):
    piece_map = {value: 0 for key, value in enumerate("KQRBNPkqrbnp")}
    # print(f"Piece_Map: {piece_map}")
    ranks = fen.split("/")
    # print(f"ranks: {ranks}")
    curr_rank = 8
    while curr_rank > 0:
        r = ranks[8-curr_rank]
        file_index = 0
        # print(f"rank: {curr_rank-1} | {r}")
        for i in range(len(r)):
            c = r[i]
            if file_index >= 8:
                convert_pieces_to_vectors(piece_map)
                states = r[i:].split(" ")[1:]
                # print(f"States: {states}")
                piece_map["moves"] = 0 if states[0] == 'w' else 1
                piece_map['WhiteKing'] = 1 if "K" in states[1] else 0
                piece_map['WhiteQueen'] = 1 if 'Q' in states[1] else 0
                piece_map['BlackKing'] = 1 if 'k' in states[1] else 0
                piece_map['BlackQueen'] = 1 if 'q' in states[1] else 0
                piece_map["en_passant"] =  np.array([(convert_to_index(states[2]) >> i) & 1 for i in range(64)], dtype=np.float32)
                piece_map["half_clock"] = int(states[3])
                piece_map["full_clock"] = int(states[4])
                return piece_map
            else:
                if c.isdigit():
                    file_index += int(c)
                else:
                    # print(f"c: {c}")
                    piece_map[c] += (1 << ((curr_rank * 8) + file_index))
                    file_index += 1
        curr_rank -= 1
    return piece_map
                


In [9]:
df['Longs'] = df['FEN'].apply(convert_fen)
df = pd.concat([df, pd.json_normalize(df['Longs'])], axis=1)
df = df.drop(['FEN', 'Longs'], axis=1)
print(df.head())

   Evaluation                                                  K  \
0         -10  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1          56  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
2          -9  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
3          52  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
4         -26  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                   Q  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
4  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                   R  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...   
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...   
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0

In [10]:
print(len(df['K']))
print(len(df['en_passant']))

809877
809877


In [11]:
# Print out the dimensions and types of all relevant columns
for col in ['K', 'Q', 'R', 'B', 'N', 'P', 'k', 'q', 'r', 'b', 'n', 'p', 'en_passant']:
    print(f"{col}: Type: {type(df[col].iloc[0])}, Shape: {np.array(df[col].iloc[0]).shape}")

# Ensure 'en_passant' is an array
df['en_passant'] = df['en_passant'].apply(lambda x: np.array(x, dtype=np.float32) if not isinstance(x, np.ndarray) else x)

# Recheck dimensions and types after conversion
for col in ['K', 'Q', 'R', 'B', 'N', 'P', 'k', 'q', 'r', 'b', 'n', 'p', 'en_passant']:
    print(f"{col}: Type: {type(df[col].iloc[0])}, Shape: {np.array(df[col].iloc[0]).shape}")


K: Type: <class 'numpy.ndarray'>, Shape: (64,)
Q: Type: <class 'numpy.ndarray'>, Shape: (64,)
R: Type: <class 'numpy.ndarray'>, Shape: (64,)
B: Type: <class 'numpy.ndarray'>, Shape: (64,)
N: Type: <class 'numpy.ndarray'>, Shape: (64,)
P: Type: <class 'numpy.ndarray'>, Shape: (64,)
k: Type: <class 'numpy.ndarray'>, Shape: (64,)
q: Type: <class 'numpy.ndarray'>, Shape: (64,)
r: Type: <class 'numpy.ndarray'>, Shape: (64,)
b: Type: <class 'numpy.ndarray'>, Shape: (64,)
n: Type: <class 'numpy.ndarray'>, Shape: (64,)
p: Type: <class 'numpy.ndarray'>, Shape: (64,)
en_passant: Type: <class 'numpy.ndarray'>, Shape: (64,)
K: Type: <class 'numpy.ndarray'>, Shape: (64,)
Q: Type: <class 'numpy.ndarray'>, Shape: (64,)
R: Type: <class 'numpy.ndarray'>, Shape: (64,)
B: Type: <class 'numpy.ndarray'>, Shape: (64,)
N: Type: <class 'numpy.ndarray'>, Shape: (64,)
P: Type: <class 'numpy.ndarray'>, Shape: (64,)
k: Type: <class 'numpy.ndarray'>, Shape: (64,)
q: Type: <class 'numpy.ndarray'>, Shape: (64,)
r: T

In [12]:
def concatenate_vectors(row):
    return np.concatenate([row['K'], row['Q'], row['R'], row['B'], row['N'], row['P'], row['k'], row['q'], row['r'], row['b'], row['n'],row['p'], row['en_passant']])

df['board_state'] = df.apply(concatenate_vectors, axis=1)

In [13]:
X = pd.concat([df['board_state'].apply(pd.Series), df[['moves', 'half_clock', 'full_clock', 
      'WhiteKing', 'WhiteQueen', 'BlackKing', 'BlackQueen']]], axis=1)
y = df['Evaluation']
nfeats = len(X.columns)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
class Chess_Dataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.float32)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [15]:
BATCH_SIZE = 16
CKPT_DIR = "CKPT"
LR = 0.001
EPOCHS = 10

In [ ]:
print(X_train)
print(y_train)

In [33]:
train_dataset = Chess_Dataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataset = Chess_Dataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [19]:
class Chess_Model(nn.Module):
    def __init__(self, nfeats):
        super(Chess_Model, self).__init__()
        self.input_layer = nn.Linear(nfeats, 128)
        self.lyr1 = nn.Linear(128, 64)
        self.lyr2 = nn.Linear(64, 32)
        self.lyr3 = nn.Linear(32, 1)
    
    def forward(self, x):
        x = torch.relu(self.input_layer(x))
        x = torch.relu(self.lyr1(x))
        x = torch.relu(self.lyr2(x))
        x = self.lyr3(x)
        return x
    
model = Chess_Model(nfeats).to(device)

In [47]:
def train(dataloader, model, optimizer, epoch):
    model.to(device)
    loss_fn = nn.MSELoss()
    model.train()
    with tqdm.tqdm(dataloader, unit="batch") as tbatch:
        for X, y in tbatch:
            X = X.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            pred = model(X)
            loss = loss_fn(pred, y.view(-1, 1))
            loss.backward()
            optimizer.step()
    torch.save(
        {
            "epoch" : epoch,
            "model_state_dict" : model.state_dict(),
            "optimizer_state_dict" : optimizer.state_dict(),
            "loss" : loss
        },
        f"{CKPT_DIR}/ckpt{epoch}.pt",
    )

In [31]:
def test(dataloader, model, dataset_name):
    loss_fn = nn.MSELoss()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            
            X = X.to(device)    
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y.view(-1, 1)).item()
            pred = torch.max(pred, 1)
            correct += (pred.argmax(1) == y.view(-1, 1)).type(torch.float32).sum().item()
    test_loss /= num_batches
    correct /= size
    print(
        f"{dataset_name} Accuracy: {(100 * correct):>0.1f}% Avg loss: {test_loss:>6f}\n")

In [42]:
def make_or_restore_model(nfeat):
    model = Chess_Model(nfeat)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    checkpoints = [
        CKPT_DIR + "/" + name
        for name in os.listdir(CKPT_DIR)
        if name[-1] == "t"
    ]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print("Restoring from", latest_checkpoint)
        ckpt = torch.load(latest_checkpoint)
        model.load_state_dict(ckpt["model_state_dict"])
        optimizer.load_state_dict(ckpt["optimizer_state_dict"])
        epoch = ckpt["epoch"]
        return model, optimizer, epoch+1
    else: 
        print("Creating new model")
        return model, optimizer, 0

In [48]:
model, optimizer, epoch_start = make_or_restore_model(nfeats)
model = model.to(device)
for e in range(epoch_start, EPOCHS):
    print()
    print("Epoch", e)
    print("-------")
    model.train()
    model.to(device)
    train(train_loader, model, optimizer, e)
    print()
    model.eval()
    test(train_loader, model, "Train")
    test(test_loader, model, "Test")

Creating new model

Epoch 0
-------


100%|██████████| 40494/40494 [01:34<00:00, 426.36batch/s]



Train Accuracy: 170.5% Avg loss: 1128798.499152

Test Accuracy: 169.6% Avg loss: 1156678.569169


Epoch 1
-------


100%|██████████| 40494/40494 [01:32<00:00, 435.55batch/s]



Train Accuracy: 170.5% Avg loss: 1093662.431133

Test Accuracy: 169.6% Avg loss: 1135997.947888


Epoch 2
-------


...


Train Accuracy: 170.5% Avg loss: 935325.916132

Test Accuracy: 169.6% Avg loss: 1032000.983103


Epoch 6
-------


100%|██████████| 40494/40494 [01:35<00:00, 425.27batch/s]



Train Accuracy: 170.5% Avg loss: 903754.673558

Test Accuracy: 169.6% Avg loss: 1030255.734862


Epoch 7
-------


100%|██████████| 40494/40494 [01:33<00:00, 431.74batch/s]


KeyboardInterrupt: 